#description


here is the second part of the project
I've separated It for three regions , mean encoding , training and ensemble, training again to get lowe loss 

*   Mean Encoding 
*   Models Training 
*   Models Ensemble 



Mean encoding  
hence we have generated most of the features but mean encoding can't be done by using the concatenated dataset so I've separated them to training ,validation, test 
getting the encodings from the training and applying them for the test set 

I've saved data set and loaded it again 

Models Training 
I've trained three models 
XGboost , Lightgbm , linear regression(after scalling)
XGboost and lightgbm have an increadible performence on the validation dataset reaches0.93 ~ 0.92 rmse score 

Models Ensembel:
I just used a straight forward one , I've got the averege of both and It was better ~91.6 
I 've searched the space to get the optimal weight of every classifier 


after making the submittion , the test score was more than one 
so I've submitted each one of them alone score of linear regression was a little better but not enough to pass , so I've decided to train xgboost with more estimators and this one was better and got 0.98 loss on test set (this what I submitted )

you can that I couldn't use the whole training set and I've downcasted a lot of the types beacuse I don't have enough memory 



#mount drive and imports

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import gc
from google.colab import drive
from sklearn.metrics import mean_squared_error


In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Reading Data ,downcasting datatypes, data split 

In [4]:

data=pd.read_csv('/content/drive/My Drive/how_to_win_data/data_lagged_tfidf_trend.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11128050 entries, 0 to 11128049
Data columns (total 62 columns):
 #   Column                                     Dtype  
---  ------                                     -----  
 0   Unnamed: 0                                 int64  
 1   date_block_num                             int64  
 2   shop_id                                    int64  
 3   item_id                                    int64  
 4   item_cnt_month                             float64
 5   city_code                                  int64  
 6   shop_name_len                              int64  
 7   shop_name_cw                               int64  
 8   shop_name_tfidf_0                          float64
 9   shop_name_tfidf_1                          float64
 10  shop_name_tfidf_2                          float64
 11  shop_name_tfidf_3                          float64
 12  shop_name_tfidf_4                          float64
 13  item_category_id                        

In [5]:
for col_name in data.columns.values:
  # print(type(data[col_name].dtype))
  if data[col_name].dtype==np.float64:
    data[col_name]=data[col_name].astype(np.float32)
data["date_block_num"]=data["date_block_num"].astype(np.int8)

data["shop_id"]=data["shop_id"].astype(np.int8)
data["item_id"]=data["item_id"].astype(np.int16)

data["city_code"]=data["city_code"].astype(np.int8)
data["shop_name_len"]=data["shop_name_len"].astype(np.int8)
data["shop_name_cw"]=data["shop_name_cw"].astype(np.int8)

data['item_category_id'] = data['item_category_id'].astype(np.int8)
data["item_name_len"]=data["item_name_len"].astype(np.int16)
data["item_name_cw"]=data["item_name_cw"].astype(np.int8)

data["type_code"]=data["type_code"].astype(np.int8)
data["subtype_code"]=data["subtype_code"].astype(np.int8)
data["item_category_name_len"]=data["item_category_name_len"].astype(np.int8)
data["item_category_name_cw"]=data["item_category_name_cw"].astype(np.int8)
data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11128050 entries, 0 to 11128049
Data columns (total 62 columns):
 #   Column                                     Dtype  
---  ------                                     -----  
 0   Unnamed: 0                                 int64  
 1   date_block_num                             int8   
 2   shop_id                                    int8   
 3   item_id                                    int16  
 4   item_cnt_month                             float32
 5   city_code                                  int8   
 6   shop_name_len                              int8   
 7   shop_name_cw                               int8   
 8   shop_name_tfidf_0                          float32
 9   shop_name_tfidf_1                          float32
 10  shop_name_tfidf_2                          float32
 11  shop_name_tfidf_3                          float32
 12  shop_name_tfidf_4                          float32
 13  item_category_id                        

In [6]:
# data.reset_index()
data.drop(['Unnamed: 0'],axis=1,inplace=True)
data.fillna(0,inplace=True)

In [7]:
gc.collect()
data.columns.values

array(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'city_code', 'shop_name_len', 'shop_name_cw', 'shop_name_tfidf_0',
       'shop_name_tfidf_1', 'shop_name_tfidf_2', 'shop_name_tfidf_3',
       'shop_name_tfidf_4', 'item_category_id', 'item_name_len',
       'item_name_cw', 'item_name_tfidf_0', 'item_name_tfidf_1',
       'item_name_tfidf_2', 'item_name_tfidf_3', 'item_name_tfidf_4',
       'type_code', 'subtype_code', 'item_category_name_len',
       'item_category_name_cw', 'item_category_name_tfidf_0',
       'item_category_name_tfidf_1', 'item_category_name_tfidf_2',
       'item_category_name_tfidf_3', 'item_category_name_tfidf_4',
       'date_avg_item_cnt_lag_1', 'date_item_avg_item_cnt_lag_1',
       'date_item_avg_item_cnt_lag_2', 'date_item_avg_item_cnt_lag_3',
       'date_item_avg_item_cnt_lag_6', 'date_item_avg_item_cnt_lag_12',
       'date_shop_avg_item_cnt_lag_1', 'date_shop_avg_item_cnt_lag_2',
       'date_shop_avg_item_cnt_lag_3', 'date_shop_avg

In [9]:
train_data=data[data.date_block_num<33]
print(train_data.columns.values)



['date_block_num' 'shop_id' 'item_id' 'item_cnt_month' 'city_code'
 'shop_name_len' 'shop_name_cw' 'shop_name_tfidf_0' 'shop_name_tfidf_1'
 'shop_name_tfidf_2' 'shop_name_tfidf_3' 'shop_name_tfidf_4'
 'item_category_id' 'item_name_len' 'item_name_cw' 'item_name_tfidf_0'
 'item_name_tfidf_1' 'item_name_tfidf_2' 'item_name_tfidf_3'
 'item_name_tfidf_4' 'type_code' 'subtype_code' 'item_category_name_len'
 'item_category_name_cw' 'item_category_name_tfidf_0'
 'item_category_name_tfidf_1' 'item_category_name_tfidf_2'
 'item_category_name_tfidf_3' 'item_category_name_tfidf_4'
 'date_avg_item_cnt_lag_1' 'date_item_avg_item_cnt_lag_1'
 'date_item_avg_item_cnt_lag_2' 'date_item_avg_item_cnt_lag_3'
 'date_item_avg_item_cnt_lag_6' 'date_item_avg_item_cnt_lag_12'
 'date_shop_avg_item_cnt_lag_1' 'date_shop_avg_item_cnt_lag_2'
 'date_shop_avg_item_cnt_lag_3' 'date_shop_avg_item_cnt_lag_6'
 'date_shop_avg_item_cnt_lag_12' 'date_cat_avg_item_cnt_lag_1'
 'date_type_avg_item_cnt_lag_1' 'date_city_avg_it

In [8]:
valid_data=data[data.date_block_num==33]
print(valid_data.columns.values)


['date_block_num' 'shop_id' 'item_id' 'item_cnt_month' 'city_code'
 'shop_name_len' 'shop_name_cw' 'shop_name_tfidf_0' 'shop_name_tfidf_1'
 'shop_name_tfidf_2' 'shop_name_tfidf_3' 'shop_name_tfidf_4'
 'item_category_id' 'item_name_len' 'item_name_cw' 'item_name_tfidf_0'
 'item_name_tfidf_1' 'item_name_tfidf_2' 'item_name_tfidf_3'
 'item_name_tfidf_4' 'type_code' 'subtype_code' 'item_category_name_len'
 'item_category_name_cw' 'item_category_name_tfidf_0'
 'item_category_name_tfidf_1' 'item_category_name_tfidf_2'
 'item_category_name_tfidf_3' 'item_category_name_tfidf_4'
 'date_avg_item_cnt_lag_1' 'date_item_avg_item_cnt_lag_1'
 'date_item_avg_item_cnt_lag_2' 'date_item_avg_item_cnt_lag_3'
 'date_item_avg_item_cnt_lag_6' 'date_item_avg_item_cnt_lag_12'
 'date_shop_avg_item_cnt_lag_1' 'date_shop_avg_item_cnt_lag_2'
 'date_shop_avg_item_cnt_lag_3' 'date_shop_avg_item_cnt_lag_6'
 'date_shop_avg_item_cnt_lag_12' 'date_cat_avg_item_cnt_lag_1'
 'date_type_avg_item_cnt_lag_1' 'date_city_avg_it

In [10]:
test_data=data[data.date_block_num==34]
print(test_data.columns.values)


['date_block_num' 'shop_id' 'item_id' 'item_cnt_month' 'city_code'
 'shop_name_len' 'shop_name_cw' 'shop_name_tfidf_0' 'shop_name_tfidf_1'
 'shop_name_tfidf_2' 'shop_name_tfidf_3' 'shop_name_tfidf_4'
 'item_category_id' 'item_name_len' 'item_name_cw' 'item_name_tfidf_0'
 'item_name_tfidf_1' 'item_name_tfidf_2' 'item_name_tfidf_3'
 'item_name_tfidf_4' 'type_code' 'subtype_code' 'item_category_name_len'
 'item_category_name_cw' 'item_category_name_tfidf_0'
 'item_category_name_tfidf_1' 'item_category_name_tfidf_2'
 'item_category_name_tfidf_3' 'item_category_name_tfidf_4'
 'date_avg_item_cnt_lag_1' 'date_item_avg_item_cnt_lag_1'
 'date_item_avg_item_cnt_lag_2' 'date_item_avg_item_cnt_lag_3'
 'date_item_avg_item_cnt_lag_6' 'date_item_avg_item_cnt_lag_12'
 'date_shop_avg_item_cnt_lag_1' 'date_shop_avg_item_cnt_lag_2'
 'date_shop_avg_item_cnt_lag_3' 'date_shop_avg_item_cnt_lag_6'
 'date_shop_avg_item_cnt_lag_12' 'date_cat_avg_item_cnt_lag_1'
 'date_type_avg_item_cnt_lag_1' 'date_city_avg_it

#mean enncoding

In [11]:
# df1.reset_index(inplace=True)
from sklearn.model_selection import KFold, StratifiedKFold
cat_features = ['shop_id', 'item_id', 'city_code', 'item_category_id', 'type_code', 'subtype_code']
temp=train_data[cat_features]
cv = KFold(n_splits=5, shuffle=False)
new_features = []

check = False
for train_idx, valid_idx in cv.split(train_data):
    
    # Train/validation split
    X_train, X_valid = train_data.iloc[train_idx,:], train_data.iloc[valid_idx,:]
    
    # Mean encoding
    for col in cat_features:
        means = X_valid[col].map(X_train.groupby(col).item_cnt_month.mean())
        col_new = col+'_target_enc'
        X_valid[col_new] = means
        
        # Results
        train_data.loc[valid_idx, col_new] = X_valid
        
        # Store new columns
        if check==False:
            new_features.append(col_new)
    
    check = True

prior = np.mean(train_data['item_cnt_month'].values)
train_data[new_features] = train_data[new_features].fillna(prior)

print(new_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

['shop_id_target_enc', 'item_id_target_enc', 'city_code_target_enc', 'item_category_id_target_enc', 'type_code_target_enc', 'subtype_code_target_enc']


In [12]:
for old_col,new_col in zip(cat_features,new_features):
        means = valid_data[old_col].map(train_data.groupby(old_col)[new_col].mean())
        valid_data[new_col] = means
        valid_data[new_col] = valid_data[new_col].fillna(prior)

        means = test_data[old_col].map(train_data.groupby(old_col)[new_col].mean())
        test_data[new_col] = means
        test_data[new_col] = test_data[new_col].fillna(prior)


valid_data.columns.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

array(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'city_code', 'shop_name_len', 'shop_name_cw', 'shop_name_tfidf_0',
       'shop_name_tfidf_1', 'shop_name_tfidf_2', 'shop_name_tfidf_3',
       'shop_name_tfidf_4', 'item_category_id', 'item_name_len',
       'item_name_cw', 'item_name_tfidf_0', 'item_name_tfidf_1',
       'item_name_tfidf_2', 'item_name_tfidf_3', 'item_name_tfidf_4',
       'type_code', 'subtype_code', 'item_category_name_len',
       'item_category_name_cw', 'item_category_name_tfidf_0',
       'item_category_name_tfidf_1', 'item_category_name_tfidf_2',
       'item_category_name_tfidf_3', 'item_category_name_tfidf_4',
       'date_avg_item_cnt_lag_1', 'date_item_avg_item_cnt_lag_1',
       'date_item_avg_item_cnt_lag_2', 'date_item_avg_item_cnt_lag_3',
       'date_item_avg_item_cnt_lag_6', 'date_item_avg_item_cnt_lag_12',
       'date_shop_avg_item_cnt_lag_1', 'date_shop_avg_item_cnt_lag_2',
       'date_shop_avg_item_cnt_lag_3', 'date_shop_avg

In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10675678 entries, 0 to 10675677
Data columns (total 67 columns):
 #   Column                                     Dtype  
---  ------                                     -----  
 0   date_block_num                             int8   
 1   shop_id                                    int8   
 2   item_id                                    int16  
 3   item_cnt_month                             float32
 4   city_code                                  int8   
 5   shop_name_len                              int8   
 6   shop_name_cw                               int8   
 7   shop_name_tfidf_0                          float32
 8   shop_name_tfidf_1                          float32
 9   shop_name_tfidf_2                          float32
 10  shop_name_tfidf_3                          float32
 11  shop_name_tfidf_4                          float32
 12  item_category_id                           int8   
 13  item_name_len                           

In [14]:
train_data.to_csv('/content/drive/My Drive/how_to_win_data/train_data.csv')
valid_data.to_csv('/content/drive/My Drive/how_to_win_data/valid_data.csv')
test_data.to_csv('/content/drive/My Drive/how_to_win_data/test_data.csv')
drive.flush_and_unmount()

#reading_data

In [3]:
train_data=pd.read_csv('/content/drive/My Drive/how_to_win_data/train_data.csv')
train_data=train_data[train_data.date_block_num>23]
for col_name in train_data.columns.values:
  # print(type(data[col_name].dtype))
  if train_data[col_name].dtype==np.float64:
    train_data[col_name]=train_data[col_name].astype(np.float32)
train_data["date_block_num"]=train_data["date_block_num"].astype(np.int8)

train_data["shop_id"]=train_data["shop_id"].astype(np.int8)
train_data["item_id"]=train_data["item_id"].astype(np.int16)

train_data["city_code"]=train_data["city_code"].astype(np.int8)
train_data["shop_name_len"]=train_data["shop_name_len"].astype(np.int8)
train_data["shop_name_cw"]=train_data["shop_name_cw"].astype(np.int8)

train_data['item_category_id'] = train_data['item_category_id'].astype(np.int8)
train_data["item_name_len"]=train_data["item_name_len"].astype(np.int16)
train_data["item_name_cw"]=train_data["item_name_cw"].astype(np.int8)

train_data["type_code"]=train_data["type_code"].astype(np.int8)
train_data["subtype_code"]=train_data["subtype_code"].astype(np.int8)
train_data["item_category_name_len"]=train_data["item_category_name_len"].astype(np.int8)
train_data["item_category_name_cw"]=train_data["item_category_name_cw"].astype(np.int8)
train_data.info()
gc.collect()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2247405 entries, 8428273 to 10675677
Data columns (total 68 columns):
 #   Column                                     Dtype  
---  ------                                     -----  
 0   Unnamed: 0                                 int64  
 1   date_block_num                             int8   
 2   shop_id                                    int8   
 3   item_id                                    int16  
 4   item_cnt_month                             float32
 5   city_code                                  int8   
 6   shop_name_len                              int8   
 7   shop_name_cw                               int8   
 8   shop_name_tfidf_0                          float32
 9   shop_name_tfidf_1                          float32
 10  shop_name_tfidf_2                          float32
 11  shop_name_tfidf_3                          float32
 12  shop_name_tfidf_4                          float32
 13  item_category_id                   

0

In [4]:
valid_data=pd.read_csv('/content/drive/My Drive/how_to_win_data/valid_data.csv')

for col_name in valid_data.columns.values:
  # print(type(data[col_name].dtype))
  if valid_data[col_name].dtype==np.float64:
    valid_data[col_name]=valid_data[col_name].astype(np.float32)
valid_data["date_block_num"]=valid_data["date_block_num"].astype(np.int8)

valid_data["shop_id"]=valid_data["shop_id"].astype(np.int8)
valid_data["item_id"]=valid_data["item_id"].astype(np.int16)

valid_data["city_code"]=valid_data["city_code"].astype(np.int8)
valid_data["shop_name_len"]=valid_data["shop_name_len"].astype(np.int8)
valid_data["shop_name_cw"]=valid_data["shop_name_cw"].astype(np.int8)

valid_data['item_category_id'] = valid_data['item_category_id'].astype(np.int8)
valid_data["item_name_len"]=valid_data["item_name_len"].astype(np.int16)
valid_data["item_name_cw"]=valid_data["item_name_cw"].astype(np.int8)

valid_data["type_code"]=valid_data["type_code"].astype(np.int8)
valid_data["subtype_code"]=valid_data["subtype_code"].astype(np.int8)
valid_data["item_category_name_len"]=valid_data["item_category_name_len"].astype(np.int8)
valid_data["item_category_name_cw"]=valid_data["item_category_name_cw"].astype(np.int8)
valid_data.info()
gc.collect()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238172 entries, 0 to 238171
Data columns (total 68 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   Unnamed: 0                                 238172 non-null  int64  
 1   date_block_num                             238172 non-null  int8   
 2   shop_id                                    238172 non-null  int8   
 3   item_id                                    238172 non-null  int16  
 4   item_cnt_month                             238172 non-null  float32
 5   city_code                                  238172 non-null  int8   
 6   shop_name_len                              238172 non-null  int8   
 7   shop_name_cw                               238172 non-null  int8   
 8   shop_name_tfidf_0                          238172 non-null  float32
 9   shop_name_tfidf_1                          238172 non-null  float32
 10  shop_nam

0

In [5]:
test_data=pd.read_csv('/content/drive/My Drive/how_to_win_data/test_data.csv')

for col_name in test_data.columns.values:
  # print(type(data[col_name].dtype))
  if test_data[col_name].dtype==np.float64:
    test_data[col_name]=test_data[col_name].astype(np.float32)
test_data["date_block_num"]=test_data["date_block_num"].astype(np.int8)

test_data["shop_id"]=test_data["shop_id"].astype(np.int8)
test_data["item_id"]=test_data["item_id"].astype(np.int16)

test_data["city_code"]=test_data["city_code"].astype(np.int8)
test_data["shop_name_len"]=test_data["shop_name_len"].astype(np.int8)
test_data["shop_name_cw"]=test_data["shop_name_cw"].astype(np.int8)

test_data['item_category_id'] = test_data['item_category_id'].astype(np.int8)
test_data["item_name_len"]=test_data["item_name_len"].astype(np.int16)
test_data["item_name_cw"]=test_data["item_name_cw"].astype(np.int8)

test_data["type_code"]=test_data["type_code"].astype(np.int8)
test_data["subtype_code"]=test_data["subtype_code"].astype(np.int8)
test_data["item_category_name_len"]=test_data["item_category_name_len"].astype(np.int8)
test_data["item_category_name_cw"]=test_data["item_category_name_cw"].astype(np.int8)
test_data.info()
gc.collect()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 68 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   Unnamed: 0                                 214200 non-null  int64  
 1   date_block_num                             214200 non-null  int8   
 2   shop_id                                    214200 non-null  int8   
 3   item_id                                    214200 non-null  int16  
 4   item_cnt_month                             214200 non-null  float32
 5   city_code                                  214200 non-null  int8   
 6   shop_name_len                              214200 non-null  int8   
 7   shop_name_cw                               214200 non-null  int8   
 8   shop_name_tfidf_0                          214200 non-null  float32
 9   shop_name_tfidf_1                          214200 non-null  float32
 10  shop_nam

0

In [6]:
X_train = train_data.drop(['item_cnt_month'], axis=1)
Y_train = train_data["item_cnt_month"]
del train_data
gc.collect()

X_valid = valid_data.drop(['item_cnt_month'], axis=1)
Y_valid = valid_data["item_cnt_month"]
del valid_data
gc.collect()

X_test = test_data.drop(['item_cnt_month'], axis=1)
del test_data
gc.collect()


0

#training

In [ ]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression, Ridge, Lasso, Lars, ElasticNet

from sklearn.externals import joblib



/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


## linear regreression

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train_std = scaler.transform(X_train)
X_valid_std = scaler.transform(X_valid)
X_test_std = scaler.transform(X_test)


In [11]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, Lars, ElasticNet

model_linreg = LinearRegression(fit_intercept=True)
model_linreg.fit(X_train_std, Y_train)
print(np.sqrt(mean_squared_error(Y_valid, model_linreg.predict(X_valid_std))))
# joblib.dump(model_linreg, '/content/drive/My Drive/how_to_win_data/models/model3_linreg.pkl')

0.9802247366938576


In [12]:
pred=model_linreg.predict(X_test_std)

temp = pd.DataFrame({'ID': X_test.index, 'item_cnt_month': pred})
temp.to_csv('/content/drive/My Drive/how_to_win_data/submission/submission_linear.csv', index=False)

## LightGBM

In [13]:
cat_features = ['shop_id', 'item_id', 'city_code', 'item_category_id', 'type_code', 'subtype_code']
# cat_features=[]
for col in cat_features[1:]:
    col_new = col+'_target_enc'
    cat_features.append(col_new)


In [17]:
lgb_train = lgb.Dataset(X_train,Y_train)
lgb_valid = lgb.Dataset(X_valid, Y_valid)

params = {
    'boosting_type': 'dart',
    'metric': 'l2_root', # RMSE
    'verbose': 1,
    'seed': 0,
    'max_depth': 15,
    'learning_rate': 0.5,
    'reg_lambda': 2.0,
    'reg_alpha': 2.0,
    'subsample': 0.7,
    'num_leaves': 20,
    'feature_fraction': 0.6,
    'drop_rate': 0.2
}

model_lgbm = lgb.train(params, lgb_train, num_boost_round=30, valid_sets=lgb_valid,
                       early_stopping_rounds=200, categorical_feature=cat_features,
                       verbose_eval=10)

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['city_code', 'city_code_target_enc', 'item_category_id', 'item_category_id_target_enc', 'item_id', 'item_id_target_enc', 'shop_id', 'subtype_code', 'subtype_code_target_enc', 'type_code', 'type_code_target_enc']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')
/usr/local/lib/python3.6/dist-packages/lightgbm/callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


[10]	valid_0's rmse: 0.954405
[20]	valid_0's rmse: 0.954337
[30]	valid_0's rmse: 0.949282


In [18]:
np.sqrt(mean_squared_error(Y_valid, model_lgbm.predict(X_valid)))

0.9532127714562435

In [19]:
pred=model_lgbm.predict(X_test)

temp = pd.DataFrame({'ID': X_test.index, 'item_cnt_month': pred})
temp.to_csv('/content/drive/My Drive/how_to_win_data/submission/submission_lgb.csv', index=False)

In [ ]:
lgb_train = lgb.Dataset(X_train,Y_train)
lgb_valid = lgb.Dataset(X_valid, Y_valid)

params = {
    'boosting_type': 'dart',
    'metric': 'l2_root', # RMSE
    'verbose': 1,
    'seed': 0,
    'max_depth': 20,
    'learning_rate': 0.5,
    'reg_lambda': 2.0,
    'reg_alpha': 2.0,
    'subsample': 0.7,
    'num_leaves': 40,
    'feature_fraction': 0.6,
    'drop_rate': 0.2
}

model_lgbm = lgb.train(params, lgb_train, num_boost_round=160, valid_sets=lgb_valid,
                       early_stopping_rounds=200, categorical_feature=cat_features,
                       verbose_eval=10)

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['city_code', 'city_code_target_enc', 'item_category_id', 'item_category_id_target_enc', 'item_id', 'item_id_target_enc', 'shop_id', 'subtype_code', 'subtype_code_target_enc', 'type_code', 'type_code_target_enc']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')
/usr/local/lib/python3.6/dist-packages/lightgbm/callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


[10]	valid_0's rmse: 0.94499
[20]	valid_0's rmse: 0.949146
[30]	valid_0's rmse: 0.946162
[40]	valid_0's rmse: 0.944231
[50]	valid_0's rmse: 0.950076
[60]	valid_0's rmse: 0.945399
[70]	valid_0's rmse: 0.945486
[80]	valid_0's rmse: 0.942773
[90]	valid_0's rmse: 0.948159
[100]	valid_0's rmse: 0.94774
[110]	valid_0's rmse: 0.94812
[120]	valid_0's rmse: 0.946723
[130]	valid_0's rmse: 0.945507
[140]	valid_0's rmse: 0.933658
[150]	valid_0's rmse: 0.930989
[160]	valid_0's rmse: 0.929449


In [ ]:
joblib.dump(model_lgbm, '/content/drive/My Drive/how_to_win_data/models/model1_lgbm.pkl');

In [ ]:
np.sqrt(mean_squared_error(Y_valid, model_lgbm.predict(X_valid)))

0.9330793160314531

In [ ]:
np.sqrt(mean_squared_error(Y_valid,(model_xgb.predict(X_valid)+model_lgbm.predict(X_valid))/2))

0.9172132316068591

## xgboost

In [ ]:
model_xgb = xgb.XGBRegressor(max_depth=8, n_estimators=1000, min_child_weight=300,
                             colsample_bytree=0.8, subsample=0.8, eta=0.2, seed=42,
                             reg_alpha=2.0, reg_lambda=2.0)

model_xgb.fit(X_train, Y_train, eval_metric='rmse', eval_set=[(X_train, Y_train),(X_valid, Y_valid)],
              verbose=True, early_stopping_rounds=1)


In [21]:
print(np.sqrt(mean_squared_error(Y_valid, model_xgb.predict(X_valid))))
pred=model_xgb.predict(X_test)

temp = pd.DataFrame({'ID': X_test.index, 'item_cnt_month': pred})
temp.to_csv('/content/drive/My Drive/how_to_win_data/submission/submission_xgb.csv', index=False)

0.9538176


In [ ]:
model_xgb = xgb.XGBRegressor(max_depth=8, n_estimators=1000, min_child_weight=300,
                             colsample_bytree=0.8, subsample=0.8, eta=0.2, seed=42,
                             reg_alpha=2.0, reg_lambda=2.0)

model_xgb.fit(X_train, Y_train, eval_metric='rmse', eval_set=[(X_valid, Y_valid)],
              verbose=True, early_stopping_rounds=100)
# joblib.dump(model_xgb, '/content/drive/My Drive/how_to_win_data/models/model2_xgbm.pkl');


[23:20:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1.12602
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:1.1012
[2]	validation_0-rmse:1.07843
[3]	validation_0-rmse:1.05757
[4]	validation_0-rmse:1.04047
[5]	validation_0-rmse:1.02701
[6]	validation_0-rmse:1.01558
[7]	validation_0-rmse:1.00662
[8]	validation_0-rmse:0.998694
[9]	validation_0-rmse:0.992809
[10]	validation_0-rmse:0.987671
[11]	validation_0-rmse:0.982822
[12]	validation_0-rmse:0.977913
[13]	validation_0-rmse:0.975259
[14]	validation_0-rmse:0.97266
[15]	validation_0-rmse:0.970584
[16]	validation_0-rmse:0.968556
[17]	validation_0-rmse:0.966844
[18]	validation_0-rmse:0.965689
[19]	validation_0-rmse:0.964658
[20]	validation_0-rmse:0.963633
[21]	validation_0-rmse:0.96317
[22]	validation_0-rmse:0.961091
[23]	validation_0-rmse:0.95999
[24]	validation_0-rmse:0.95871
[25]	validation_0-rmse:0.95853

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.2, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=300, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=2.0, reg_lambda=2.0, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.8, verbosity=1)

In [ ]:
model_xgb = joblib.load('/content/drive/My Drive/how_to_win_data/models/model2_xgbm.pkl') 

[02:06:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
print(np.sqrt(mean_squared_error(Y_valid, model_xgb.predict(X_valid))))

0.9370115


In [ ]:
del model_xgb

In [ ]:
gc.collect()

219

# ensembling

In [ ]:
model_lgbm =joblib.load('/content/drive/My Drive/how_to_win_data/models/model1_lgbm.pkl')
model_xgb = joblib.load('/content/drive/My Drive/how_to_win_data/models/model2_xgbm.pkl') 
model_linreg=joblib.load('/content/drive/My Drive/how_to_win_data/models/model3_linreg.pkl')
X_valid_std = scaler.transform(X_valid)


[21:43:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
a=model_xgb.predict(X_valid)
b=model_lgbm.predict(X_valid)



In [ ]:
print(np.sqrt(mean_squared_error(Y_valid,(a+b)/2)))

0.9165924431094229


In [ ]:
# 500*a+500*b/2
print(np.sqrt(mean_squared_error(Y_valid,(500*a+500*b)/1000)))

0.916592443031179


In [ ]:
min_error=np.sqrt(mean_squared_error(Y_valid,(a+b)/2))
for i in range(1000):
  error=np.sqrt(mean_squared_error(Y_valid,(i*a+(1000-i)*b)/1000))
  if error<min_error:
    print(i,'  ',error)
    min_error=error



445    0.9165898534584336
446    0.9165859915582303
447    0.9165822746447747
448    0.9165787021130413
449    0.9165752745230846
450    0.9165719916746445
451    0.9165688537146838
452    0.9165658603141733
453    0.9165630118705083
454    0.916560308125568
455    0.9165577492586866
456    0.9165553349609619
457    0.9165530655771438
458    0.9165509410082343
459    0.9165489611767859
460    0.9165471259738078
461    0.9165454357161036
462    0.9165438900648477
463    0.9165424893724697
464    0.9165412335450869
465    0.9165401222248671
466    0.9165391558753884
467    0.9165383343360872
468    0.9165376574928141
469    0.9165371253690331
470    0.916536738108025
471    0.9165364957419462
472    0.9165363979205506


In [ ]:
c=(472*a+(1000-472)*b)/1000

In [ ]:
np.sqrt(mean_squared_error(Y_valid,c)

0.9165363979205506

In [ ]:
temp = pd.DataFrame({'ID': X_test.index, 'item_cnt_month':c})
temp.to_csv('/content/drive/My Drive/how_to_win_data/submission/submission.csv', index=False)

In [ ]:
temp = pd.DataFrame({'ID': X_test.index, 'item_cnt_month': a})
temp.to_csv('/content/drive/My Drive/how_to_win_data/submission/submission1.csv', index=False)

In [ ]:
temp = pd.DataFrame({'ID': X_test.index, 'item_cnt_month': b})
temp.to_csv('/content/drive/My Drive/how_to_win_data/submission/submission2.csv', index=False)

#training again

In [7]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression, Ridge, Lasso, Lars, ElasticNet

from sklearn.externals import joblib



/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


## xgboost

In [8]:
model_xgb = xgb.XGBRegressor(max_depth=8, n_estimators=1500, min_child_weight=300,
                             colsample_bytree=0.8, subsample=0.5, eta=0.2, seed=42,
                             reg_alpha=2.0, reg_lambda=2.0)

model_xgb.fit(X_train, Y_train, eval_metric='rmse', eval_set=[(X_valid, Y_valid)],
              verbose=True, early_stopping_rounds=1)
print(np.sqrt(mean_squared_error(Y_valid, model_xgb.predict(X_valid))))
pred=model_xgb.predict(X_test)

temp = pd.DataFrame({'ID': X_test.index, 'item_cnt_month': pred})
temp.to_csv('/content/drive/My Drive/how_to_win_data/submission/submission_xgb_new1.csv', index=False)
gc.collect()

[17:26:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1.12473
Will train until validation_0-rmse hasn't improved in 1 rounds.
[1]	validation_0-rmse:1.09284
[2]	validation_0-rmse:1.06746
[3]	validation_0-rmse:1.04488
[4]	validation_0-rmse:1.02798
[5]	validation_0-rmse:1.01316
[6]	validation_0-rmse:1.00042
[7]	validation_0-rmse:0.98866
[8]	validation_0-rmse:0.978973
[9]	validation_0-rmse:0.969322
[10]	validation_0-rmse:0.963148
[11]	validation_0-rmse:0.957771
[12]	validation_0-rmse:0.951293
[13]	validation_0-rmse:0.947241
[14]	validation_0-rmse:0.944028
[15]	validation_0-rmse:0.942183
[16]	validation_0-rmse:0.939905
[17]	validation_0-rmse:0.936702
[18]	validation_0-rmse:0.935131
[19]	validation_0-rmse:0.933098
[20]	validation_0-rmse:0.931963
[21]	validation_0-rmse:0.930152
[22]	validation_0-rmse:0.928741
[23]	validation_0-rmse:0.927756
[24]	validation_0-rmse:0.92675
[25]	validation_0-rmse:0.925

0

## linear regression

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train_std = scaler.transform(X_train)
X_valid_std = scaler.transform(X_valid)
X_test_std = scaler.transform(X_test)
gc.collect()

0

In [9]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, Lars, ElasticNet

model_linreg = LinearRegression(fit_intercept=True)
model_linreg.fit(X_train_std, Y_train)
print(np.sqrt(mean_squared_error(Y_valid, model_linreg.predict(X_valid_std))))
pred=model_linreg.predict(X_test_std)

temp = pd.DataFrame({'ID': X_test.index, 'item_cnt_month': pred})
temp.to_csv('/content/drive/My Drive/how_to_win_data/submission/submission_linear_new.csv', index=False)
gc.collect()

0.9686693410397219


0